# Loading necessary libraries

In [9]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.models import Sequential, load_model
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder

from keras.layers import Conv2D, MaxPooling2D, Dense, Activation, Dropout, Flatten, BatchNormalization
from keras.optimizers import Adam
from keras.initializers import  RandomNormal
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
%matplotlib inline
from tensorflow.keras.preprocessing import image

import pandas as pd

from skimage import feature
from skimage.feature import local_binary_pattern
from skimage.color import rgb2gray

from keras.layers import Input, concatenate
from keras.models import Model

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split

# Checking if GPU is available
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Loading and preprocessing images

In [10]:
# Fungsi untuk menghitung LBP pada satu gambar
def calculate_lbp_image(img):
    lbp_image = local_binary_pattern(img, 8, 1, method='uniform')
    return lbp_image

In [11]:
data = pd.read_csv("../fer2013.csv")

# Building LBP

In [12]:
# Ekstraksi fitur LBP dan simpan dalam list
lbp_features = []
for i in range(len(data)):
    pixels = np.array(data['pixels'].iloc[i].split(' '), dtype=int)
    img = pixels.reshape(48, 48)
    lbp_image = calculate_lbp_image(img)
    lbp_features.append(lbp_image)

In [13]:
# Konversi list ke array
lbp_features = np.array(lbp_features)

In [14]:
# Konversi label emosi menjadi one-hot encoding
encoder = LabelEncoder()
labels = encoder.fit_transform(data['emotion'])
labels = tf.keras.utils.to_categorical(labels, num_classes=7)  # Ubah menjadi 7 karena ada 7 emosi dalam dataset

In [15]:
# Bagi data menjadi set pelatihan dan validasi
X_train, X_val, y_train, y_val = train_test_split(lbp_features, labels, test_size=0.2, random_state=42)

# Building CNN

In [16]:
# Constructing CNN structure

# VGG16 arsitektur
# Membuat model VGG16 dengan citra grayscale
model = Sequential()

# Layer 1 (grayscale input)
model.add(Conv2D(64, (3, 3), input_shape=(48, 48, 1), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Layer 2
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Layer 3
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Layer 4
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Layer 5
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Fully connected layers
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))  # 7 classes for ImageNet


In [17]:
# Tampilkan ringkasan model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 64)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 128)      0         
 2D)                                                    

# Training CNN

In [14]:
# # Compiling model
# model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# # Specifying parameters for Data Augmentation
# datagen = ImageDataGenerator(
#     featurewise_center=False,  # set input mean to 0 over the dataset
#     samplewise_center=False,  # set each sample mean to 0
#     featurewise_std_normalization=False,  # divide inputs by std of the dataset
#     samplewise_std_normalization=False,  # divide each input by its std
#     zca_whitening=False,  # apply ZCA whitening
#     rotation_range=40,  # randomly rotate images in the range (degrees, 0 to 180)
#     width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
#     height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
#     horizontal_flip=True,  # randomly flip images
#     vertical_flip=False,
#     zoom_range = 0.05)  # zoom images in range [1 - zoom_range, 1+ zoom_range]

# datagen.fit(X_train)

# history = model.fit_generator(datagen.flow(X_train, y_train,
#                     batch_size=32),
#                     epochs=10,  # Mengganti nb_epoch menjadi epochs
#                     validation_data=(X_val, y_val),
#                     steps_per_epoch=X_train.shape[0] // 32,  # Menggunakan // untuk pembagian bulat
#                     )

# # pd.DataFrame(history.history).to_csv("history.csv")

C:\Users\hp\.conda\envs\tensorflowgpu\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/50
455/455 [==============================] - 43s 83ms/step - loss: 2.0082 - accuracy: 0.2468 - val_loss: 1.7945 - val_accuracy: 0.2517
Epoch 2/50
455/455 [==============================] - 36s 80ms/step - loss: 1.7948 - accuracy: 0.2530 - val_loss: 1.7823 - val_accuracy: 0.2622
Epoch 3/50
455/455 [==============================] - 37s 81ms/step - loss: 1.7470 - accuracy: 0.2828 - val_loss: 1.7161 - val_accuracy: 0.2960
Epoch 4/50
455/455 [==============================] - 37s 81ms/step - loss: 1.6110 - accuracy: 0.3674 - val_loss: 1.5898 - val_accuracy: 0.3718
Epoch 5/50
455/455 [==============================] - 37s 81ms/step - loss: 1.4380 - accuracy: 0.4503 - val_loss: 1.6102 - val_accuracy: 0.3783
Epoch 6/50
455/455 [==============================] - 37s 81ms/step - loss: 1.1193 - accuracy: 0.5813 - val_loss: 1.7620 - val_accuracy: 0.3737
Epoch 7/50
455/455 [==============================] - 37s 81ms/step - loss: 0.6937 - accuracy: 0.7522 - val_loss: 2.0751 - val_accuracy:

In [19]:
# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Reshape data LBP untuk memenuhi input model
X_train = X_train.reshape(-1, 48, 48, 1)
X_val = X_val.reshape(-1, 48, 48, 1)

# Latih model dengan data LBP
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=64)


Epoch 1/100
449/449 [==============================] - 34s 72ms/step - loss: 1.8126 - accuracy: 0.2516 - val_loss: 1.8102 - val_accuracy: 0.2459
Epoch 2/100
449/449 [==============================] - 33s 73ms/step - loss: 1.8126 - accuracy: 0.2516 - val_loss: 1.8102 - val_accuracy: 0.2459
Epoch 3/100
449/449 [==============================] - 33s 73ms/step - loss: 1.8124 - accuracy: 0.2516 - val_loss: 1.8102 - val_accuracy: 0.2459
Epoch 4/100
449/449 [==============================] - 33s 73ms/step - loss: 1.8116 - accuracy: 0.2516 - val_loss: 1.8113 - val_accuracy: 0.2459
Epoch 5/100
449/449 [==============================] - 33s 74ms/step - loss: 1.8117 - accuracy: 0.2516 - val_loss: 1.8096 - val_accuracy: 0.2459
Epoch 6/100
449/449 [==============================] - 33s 74ms/step - loss: 1.8124 - accuracy: 0.2516 - val_loss: 1.8097 - val_accuracy: 0.2459
Epoch 7/100
449/449 [==============================] - 33s 74ms/step - loss: 1.8122 - accuracy: 0.2516 - val_loss: 1.8105 - val_ac

KeyboardInterrupt: 

In [16]:
# Simpan model ke dalam file
# final_model.save("LBPVGG16.h5")

# Loading best model and exploring the results

In [17]:
model = load_model("LBPVGG16.h5")

In [18]:
final_model.evaluate([X_train, lbp_images_train], y_train)

909/909 [==============================] - 77s 84ms/step - loss: 0.0242 - accuracy: 0.9961


[0.024201760068535805, 0.9960781335830688]

In [20]:
model.evaluate([X_train, lbp_images_train], y_train)

909/909 [==============================] - 62s 68ms/step - loss: 0.0242 - accuracy: 0.9961


[0.024201760068535805, 0.9960781335830688]

In [21]:
test_evaluation = final_model.evaluate([X_test, lbp_images_test], y_test)
train_evaluation = final_model.evaluate([X_train, lbp_images_train], y_train)

909/909 [==============================] - 326s 358ms/step - loss: 0.0242 - accuracy: 0.9961


In [23]:
test_evaluation = model.evaluate([X_test, lbp_images_test], y_test)
train_evaluation = model.evaluate([X_train, lbp_images_train], y_train)

909/909 [==============================] - 293s 322ms/step - loss: 0.0242 - accuracy: 0.9961


In [ ]:
def predict_classes(model, test_imgs, test_labels, emotions_dict,  batch_size  = 32):    

    # Predict class of image using trained model
    class_pred = model.predict(test_imgs, batch_size = batch_size)

    # Convert vector of zeros and ones to label
    labels_pred = np.argmax(class_pred,axis=1)
    true_labels = np.argmax(test_labels, axis=1)

    # Boolean array that indicates whether the predicted label is the true label
    correct = labels_pred == true_labels
    
    # Converting array of labels into emotion names
    pred_emotion_names = pd.Series(labels_pred).map(emotions_dict)
    
    results = {'Predicted_label': labels_pred, 'Predicted_emotion': pred_emotion_names, 'Is_correct' : correct}
    results = pd.DataFrame(results)
    return correct, results


def visualize_predictions(images_test, orglabel_names, predlabel_names, correct_arr, valid = True):
    
    if valid == True:
        correct = np.array(np.where(correct_arr == True))[0]
        # Plot 15 randomly selected and correctly predicted images
        show_random(images_test, emotion_nms_org = orglabel_names, emotion_nms_pred = predlabel_names, random = False, indices = correct)
    else:
        incorrect = np.array(np.where(correct_arr == False))[0]
        # Plot 15 randomly selected and wrongly predicted images
        show_random(images_test, emotion_nms_org = orglabel_names, emotion_nms_pred = predlabel_names, random = False, indices = incorrect)
 

In [ ]:
correct, results_df = predict_classes(model, X_test, y_test, emotions_names, batch_size = 64)
results_df['Original_label'] = data['emotion'][32298:].values
results_df['True_emotion'] = results_df['Original_label'].map(emotions_names)

### Correctly predicted images

In [ ]:
def visualize_predictions(images, true_emotions, predicted_emotions, correct, valid=True):
    plt.figure(figsize=(12, 12))
    count = 0
    
    for i in range(len(images)):
        if correct[i] == valid:  # Filter based on whether prediction is correct or not
            count += 1
            # Reshape the flattened image to (48, 48)
            reshaped_image = images[i].reshape(48, 48)
            plt.subplot(5, 5, count)
            plt.imshow(reshaped_image, cmap='gray')
            plt.title(f'True: {true_emotions[i]}\nPredicted: {predicted_emotions[i]}')
            plt.axis('off')
            
            if count >= 25:  # Display up to 25 images
                break
    
    plt.show()

# Call the function to visualize incorrect predictions
visualize_predictions(images_test, results_df['True_emotion'], results_df['Predicted_emotion'], correct, valid=True)


### Wronly predicted images

In [ ]:
def visualize_predictions(images, true_emotions, predicted_emotions, correct, valid=True):
    plt.figure(figsize=(12, 12))
    count = 0
    
    for i in range(len(images)):
        if correct[i] == valid:  # Filter based on whether prediction is correct or not
            count += 1
            # Reshape the flattened image to (48, 48)
            reshaped_image = images[i].reshape(48, 48)
            plt.subplot(5, 5, count)
            plt.imshow(reshaped_image, cmap='gray')
            plt.title(f'True: {true_emotions[i]}\nPredicted: {predicted_emotions[i]}')
            plt.axis('off')
            
            if count >= 25:  # Display up to 25 images
                break
    
    plt.show()

# Call the function to visualize incorrect predictions
visualize_predictions(images_test, results_df['True_emotion'], results_df['Predicted_emotion'], correct, valid=False)


### Confusion matrix

In [ ]:
def create_confmat(true_labels, predicted_labels, columns, colour = 'Oranges', size = (20,14)):
    
    cm = confusion_matrix(true_labels, predicted_labels) 
    cm_df = pd.DataFrame(cm,
    index = [col for col in columns], 
    columns = [col for col in columns])
    plt.figure(figsize=(18,16))
    sns.heatmap(cm_df, annot = True, cmap = colour, fmt='g', linewidths=.2)
    plt.title('Confusion Matrix', fontsize = 20)
    plt.ylabel('True label', fontsize = 18)
    plt.xlabel('Predicted label', fontsize = 18)
    plt.tick_params(axis='both', labelsize=14)
    plt.show()

In [ ]:
# Displaying confusion matrix
create_confmat(results_df['Original_label'], results_df['Predicted_label'], ['Angry','Disgust','Fear', 'Happy', 'Sad', 'Surprise', 'Neutral'], colour = 'Greens')

In [ ]:
# Path ke gambar
img_path = 'D:/Semester_7/FER-2013/test/fear/PrivateTest_2632530.jpg'

# Memuat gambar ke dalam bentuk grayscale
test_image = image.load_img(img_path, target_size=(48, 48), color_mode='grayscale')
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image = test_image / 255.0

# Melakukan prediksi
result = model.predict(test_image)
predicted_class = np.argmax(result[0])

emotions_names = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}
print(emotions_names)

# Menampilkan gambar
plt.imshow(test_image[0, :, :, 0], cmap='gray')  # Munculkan gambar grayscale
plt.title('Predicted class: {}'.format(predicted_class))
plt.show()